In [ ]:
!pip install --upgrade matplotlib seaborn

In [ ]:
import numpy as np
import tensorflow as tf
import datetime as dt
import os
from tqdm.notebook import tqdm
from google.colab import drive
import json
import seaborn as sns
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import random
import scipy.stats as stats

In [ ]:
import tensorflow as tf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import os
from tqdm import tqdm
from google.colab import drive
from scipy.stats import norm

# ==================== DATA GENERATION ====================
def monte_carlo_paths(S_0, time_to_expiry, sigma, drift, seed, n_sims, n_timesteps):
    """Génère des paths de prix en 3D (timesteps+1, n_sims, 1)"""
    if seed is not None:
        np.random.seed(seed)

    dt_val = time_to_expiry / n_timesteps
    paths = np.zeros((n_timesteps + 1, n_sims, 1))
    paths[0, :, 0] = S_0

    for t in range(1, n_timesteps + 1):
        rand = np.random.normal(0, 1, n_sims)
        paths[t, :, 0] = paths[t-1, :, 0] * np.exp(
            (drift - 0.5*sigma**2)*dt_val +
            sigma*np.sqrt(dt_val)*rand
        )

    return paths

# ==================== NEURAL AGENT ====================
class Agent(tf.keras.Model):
    def __init__(self, time_steps, batch_size, features, T, r=0.05, nodes=[64, 48, 32, 1], name='model'):
        super().__init__(name=name)

        self.time_steps = time_steps
        self.batch_size = batch_size
        self.features = features
        self.T = T
        self.r = r

        # Architecture LSTM
        self.lstm_layers = [
            tf.keras.layers.LSTM(units,
                               return_sequences=True,
                               activation='tanh',
                               kernel_initializer='glorot_uniform')
            for units in nodes
        ]

        self.optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001,
            clipvalue=1.0
        )

    def call(self, inputs):
        x = inputs
        for lstm in self.lstm_layers:
            x = lstm(x)
        return tf.squeeze(x, axis=-1)

    def calculate_hedging_pnl(self, S_t_input, K, strategy):
        """Calcule le PnL avec gestion explicite du cash"""
        S = tf.squeeze(S_t_input, axis=-1)
        time_steps = tf.shape(S)[0]
        dt_val = self.T / tf.cast(time_steps - 1, tf.float32)

        # Calcul des différences de delta
        delta_diff = tf.concat([
            strategy[0:1, :],  # Position initiale
            strategy[1:, :] - strategy[:-1, :]
        ], axis=0)

        # Simulation du cash
        cash = -delta_diff[0] * S[0]
        for t in range(1, time_steps-1):
            cash = cash * tf.exp(self.r * dt_val) - delta_diff[t] * S[t]

        # Valeur finale
        cash_final = cash * tf.exp(self.r * dt_val)
        Pi_T = strategy[-1] * S[-1] + cash_final
        return Pi_T - tf.maximum(S[-1] - K, 0)

    def calculate_cvar(self, pnl, alpha):
        """Calcule le CVaR différenciable"""
        sorted_pnl = tf.sort(pnl)
        n = tf.cast(tf.shape(pnl)[0], tf.float32)
        var_index = tf.cast((1 - alpha) * n, tf.int32)
        return -tf.reduce_mean(sorted_pnl[:var_index])

    @tf.function
    def train_step(self, S_t_input, K, alpha):
        """Step d'entraînement optimisé"""
        with tf.GradientTape() as tape:
            strategy = self(S_t_input[:-1])
            pnl = self.calculate_hedging_pnl(S_t_input, K, strategy)
            cvar = self.calculate_cvar(pnl, alpha)

        grads = tape.gradient(cvar, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return cvar, pnl, strategy

    def _execute_graph_batchwise(self, paths, strikes, riskaversion, epochs=1, training=False):
        """Gestion des batches avec suivi de progression"""
        sample_size = paths.shape[1]
        idx = np.arange(sample_size)

        pnls = []
        strategies = []

        for epoch in tqdm(range(epochs), desc='Epochs'):
            if training:
                np.random.shuffle(idx)

            for i in range(0, sample_size, self.batch_size):
                batch_idx = idx[i:i+self.batch_size]
                batch = paths[:, batch_idx, :]

                if training:
                    _, pnl, strat = self.train_step(
                        tf.cast(batch, tf.float32),
                        tf.cast(strikes[batch_idx], tf.float32),
                        tf.constant(riskaversion, tf.float32)
                    )
                else:
                    strat = self(tf.cast(batch[:-1], tf.float32))
                    pnl = self.calculate_hedging_pnl(
                        tf.cast(batch, tf.float32),
                        tf.cast(strikes[batch_idx], tf.float32),
                        strat
                    )

                pnls.append(pnl.numpy())
                strategies.append(strat.numpy())

            if training and (epoch % 10 == 0):
                current_cvar = np.mean(-np.sort(np.concatenate(pnls))[:int(0.05*sample_size)])
                print(f"Epoch {epoch} | CVaR: {current_cvar:.4f}")

        return np.concatenate(pnls), np.concatenate(strategies, axis=1)

    def training(self, paths, strikes, riskaversion, epochs):
        """Interface d'entraînement"""
        return self._execute_graph_batchwise(paths, strikes, riskaversion, epochs, training=True)

    def predict(self, paths, strikes, riskaversion):
        """Interface de prédiction"""
        return self._execute_graph_batchwise(paths, strikes, riskaversion, training=False)

# ==================== TRAINING EXECUTION ====================
def run_training():
    # Configuration
    params = {
        'S_0': 100,
        'T': 1/12,
        'r': 0.05,
        'vol': 0.2,
        'timesteps': 15,
        'n_sims': 10000,
        'batch_size': 500,
        'epochs': 10,
        'alpha': 0.95
    }

    # Génération des données
    print("Génération des paths...")
    paths = monte_carlo_paths(
        params['S_0'],
        params['T'],
        params['vol'],
        params['r'],
        seed=42,
        n_sims=params['n_sims'],
        n_timesteps=params['timesteps']
    )

    # Initialisation modèle
    model = Agent(
        time_steps=paths.shape[0],
        batch_size=params['batch_size'],
        features=1,
        T=params['T'],
        r=params['r']
    )

    # Entraînement
    print("Début de l'entraînement...")
    model.training(
        paths,
        np.full(params['n_sims'], 100),  # K=100
        params['alpha'],
        params['epochs']
    )

    return model

# ==================== TESTING FRAMEWORK ====================
class HedgingTest:
    def __init__(self, S_0=100, K=100, r=0.05, T=1/12, vol=0.2, timesteps=15):
        self.params = {
            'S_0': S_0,
            'K': K,
            'r': r,
            'T': T,
            'vol': vol,
            'timesteps': timesteps
        }
        self.dt = T / timesteps

    def black_scholes_delta(self, S, t):
        """Delta BS avec taux sans risque (version corrigée)"""
        if t <= 1e-6:
            return np.where(S > self.params['K'], 1.0, 0.0)

        # Formule correctement formatée
        d1 = (
            np.log(S/self.params['K'])
            + (self.params['r'] + 0.5*self.params['vol']**2)*t
        ) / (self.params['vol']*np.sqrt(t))

        return norm.cdf(d1)


    def calculate_bs_pnl(self, paths):
        """Calcul précis du PnL BS avec cash"""
        n_sims = paths.shape[1]
        pnls = np.zeros(n_sims)

        for i in range(n_sims):
            path = paths[:, i, 0]
            cash = 0.0
            delta_prev = 0.0

            for t in range(len(path)-1):
                time_left = self.params['T'] - t*self.dt
                delta = self.black_scholes_delta(path[t], time_left)
                cash = cash * np.exp(self.params['r'] * self.dt) - (delta - delta_prev)*path[t]
                delta_prev = delta

            cash_final = cash * np.exp(self.params['r'] * self.dt)
            Pi_T = delta_prev * path[-1] + cash_final
            pnls[i] = Pi_T - max(path[-1] - self.params['K'], 0)

        return pnls

    def compare_strategies(self, model, n_paths=5000):
    """Comparaison complète des performances"""
    # Paramètres pour la génération de paths
    path_params = {
        'S_0': self.params['S_0'],
        'time_to_expiry': self.params['T'],
        'sigma': self.params['vol'],
        'drift': self.params['r'],
        'n_timesteps': self.params['timesteps'],
        'seed': 42,
        'n_sims': n_paths
    }

    # Génération des paths
    paths = monte_carlo_paths(**path_params)

        # Calcul BS
        bs_pnl = self.calculate_bs_pnl(paths)

        # Calcul LSTM
        lstm_pnl, _ = model.predict(
            paths,
            np.full(n_paths, self.params['K']),
            0.95
        )

        # Analyse
        results = {
            'BS': {
                'Mean': bs_pnl.mean(),
                'Std': bs_pnl.std(),
                'CVaR_95': self._calculate_cvar(bs_pnl, 0.95)
            },
            'LSTM': {
                'Mean': lstm_pnl.mean(),
                'Std': lstm_pnl.std(),
                'CVaR_95': self._calculate_cvar(lstm_pnl, 0.95)
            }
        }

        # Visualisation
        self._plot_distributions(bs_pnl, lstm_pnl)
        return results

    def _calculate_cvar(self, pnl, alpha):
        sorted_pnl = np.sort(pnl)
        n_worst = int((1 - alpha) * len(pnl))
        return -sorted_pnl[:n_worst].mean()

    def _plot_distributions(self, bs_pnl, lstm_pnl):
        plt.figure(figsize=(15, 6))

        plt.subplot(1, 2, 1)
        sns.histplot(bs_pnl, color='blue', label='Black-Scholes', kde=True, alpha=0.3)
        sns.histplot(lstm_pnl, color='red', label='LSTM', kde=True, alpha=0.3)
        plt.title('Distribution des PnL')
        plt.xlabel('PnL')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.scatter(np.sort(bs_pnl), np.sort(lstm_pnl), alpha=0.5)
        plt.plot([bs_pnl.min(), bs_pnl.max()], [bs_pnl.min(), bs_pnl.max()], 'r--')
        plt.title('Q-Q Plot des PnL')
        plt.xlabel('Black-Scholes')
        plt.ylabel('LSTM')

        plt.tight_layout()
        plt.show()

# ==================== EXECUTION ====================
if __name__ == "__main__":
    # Montage Google Drive
    drive.mount('/content/drive')

    # Entraînement
    model = run_training()

    # Évaluation
    tester = HedgingTest()
    results = tester.compare_strategies(model)

    # Affichage résultats
    print("\nRésultats comparatifs:")
    print(pd.DataFrame(results).T)

IndentationError: expected an indented block after function definition on line 247 (<ipython-input-2-5688e97665af>, line 248)

In [ ]:
import tensorflow as tf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import os
from tqdm import tqdm
from google.colab import drive
from scipy.stats import norm

# ==================== GÉNÉRATION DE DONNÉES ====================
def monte_carlo_paths(S_0, time_to_expiry, sigma, drift, seed, n_sims, n_timesteps):
    """Génère des paths de prix en 3D (n_timesteps+1, n_sims, 1)"""
    if seed is not None:
        np.random.seed(seed)

    dt_val = time_to_expiry / n_timesteps
    paths = np.zeros((n_timesteps + 1, n_sims, 1))
    paths[0, :, 0] = S_0

    for t in range(1, n_timesteps + 1):
        rand = np.random.normal(0, 1, n_sims)
        paths[t, :, 0] = paths[t-1, :, 0] * np.exp(
            (drift - 0.5*sigma**2)*dt_val +
            sigma*np.sqrt(dt_val)*rand
        )

    return paths

# ==================== MODÈLE LSTM ====================
class Agent(tf.keras.Model):
    def __init__(self, time_steps, batch_size, features, T, r=0.05, nodes=[64, 48, 32, 1], name='model'):
        super().__init__(name=name)

        self.time_steps = time_steps
        self.batch_size = batch_size
        self.features = features
        self.T = T  # Maturité
        self.r = r   # Taux sans risque

        # Architecture LSTM
        self.lstm_layers = [
            tf.keras.layers.LSTM(units,
                               return_sequences=True,
                               activation='tanh',
                               kernel_initializer='glorot_uniform')
            for units in nodes
        ]

        self.optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001,
            clipvalue=1.0
        )

    def call(self, inputs):
        x = inputs
        for lstm in self.lstm_layers:
            x = lstm(x)
        return tf.squeeze(x, axis=-1)

    def calculate_hedging_pnl(self, S_t_input, K, strategy):
        """Calcule le PnL avec gestion du cash"""
        S = tf.squeeze(S_t_input, axis=-1)
        time_steps = tf.shape(S)[0]
        dt_val = self.T / tf.cast(time_steps - 1, tf.float32)

        # Calcul des différences de delta
        delta_diff = tf.concat([
            strategy[0:1, :],  # Position initiale
            strategy[1:, :] - strategy[:-1, :]
        ], axis=0)

        # Simulation du cash
        cash = -delta_diff[0] * S[0]
        for t in range(1, time_steps-1):
            cash = cash * tf.exp(self.r * dt_val) - delta_diff[t] * S[t]

        # Valeur finale
        cash_final = cash * tf.exp(self.r * dt_val)
        Pi_T = strategy[-1] * S[-1] + cash_final
        return Pi_T - tf.maximum(S[-1] - K, 0)

    def calculate_cvar(self, pnl, alpha):
        """Calcule le CVaR différenciable"""
        sorted_pnl = tf.sort(pnl)
        n = tf.cast(tf.shape(pnl)[0], tf.float32)
        var_index = tf.cast((1 - alpha) * n, tf.int32)
        return -tf.reduce_mean(sorted_pnl[:var_index])

    @tf.function
    def train_step(self, S_t_input, K, alpha):
        """Étape d'entraînement optimisée"""
        with tf.GradientTape() as tape:
            strategy = self(S_t_input[:-1])
            pnl = self.calculate_hedging_pnl(S_t_input, K, strategy)
            cvar = self.calculate_cvar(pnl, alpha)

        grads = tape.gradient(cvar, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return cvar, pnl, strategy

    def training(self, paths, strikes, riskaversion, epochs):
        """Processus d'entraînement complet"""
        sample_size = paths.shape[1]
        idx = np.arange(sample_size)

        for epoch in tqdm(range(epochs), desc='Entraînement'):
            np.random.shuffle(idx)
            pnls = []

            for i in range(0, sample_size, self.batch_size):
                batch_idx = idx[i:i+self.batch_size]
                batch = paths[:, batch_idx, :]

                _, pnl, _ = self.train_step(
                    tf.cast(batch, tf.float32),
                    tf.cast(strikes[batch_idx], tf.float32),
                    tf.constant(riskaversion, tf.float32)
                )
                pnls.append(pnl.numpy())

            if epoch % 10 == 0:
                current_cvar = np.mean(-np.sort(np.concatenate(pnls))[:int(0.05*sample_size)])
                print(f"Epoch {epoch} | CVaR: {current_cvar:.4f}")

        return self

# ==================== ÉVALUATION ====================
class HedgingTest:
    def __init__(self, S_0=100, K=100, r=0.05, T=1/12, vol=0.2, timesteps=15):
        self.params = {
            'S_0': S_0,
            'K': K,
            'r': r,
            'T': T,
            'vol': vol,
            'timesteps': timesteps
        }
        self.dt = T / timesteps

    def black_scholes_delta(self, S, t):
        """Calcul du delta Black-Scholes corrigé"""
        if t <= 1e-6:
            return np.where(S > self.params['K'], 1.0, 0.0)

        d1 = (np.log(S/self.params['K'])
              + (self.params['r'] + 0.5*self.params['vol']**2)*t
             ) / (self.params['vol']*np.sqrt(t))

        return norm.cdf(d1)

    def calculate_bs_pnl(self, paths):
        """Calcul précis du PnL BS avec cash"""
        n_sims = paths.shape[1]
        pnls = np.zeros(n_sims)

        for i in range(n_sims):
            path = paths[:, i, 0]
            cash = 0.0
            delta_prev = 0.0

            for t in range(len(path)-1):
                time_left = self.params['T'] - t*self.dt
                delta = self.black_scholes_delta(path[t], time_left)
                cash = cash * np.exp(self.params['r'] * self.dt) - (delta - delta_prev)*path[t]
                delta_prev = delta

            cash_final = cash * np.exp(self.params['r'] * self.dt)
            Pi_T = delta_prev * path[-1] + cash_final
            pnls[i] = Pi_T - max(path[-1] - self.params['K'], 0)

        return pnls

    def compare_strategies(self, model, n_paths=5000):
        """Comparaison des stratégies corrigée"""
        # Génération des paths
        path_params = {
            'S_0': self.params['S_0'],
            'time_to_expiry': self.params['T'],
            'sigma': self.params['vol'],
            'drift': self.params['r'],
            'seed': 42,
            'n_sims': n_paths,
            'n_timesteps': self.params['timesteps']
        }
        paths = monte_carlo_paths(**path_params)

        # Calcul BS
        bs_pnl = self.calculate_bs_pnl(paths)

        # Calcul LSTM
        strategy = model(paths[:-1])
        lstm_pnl = model.calculate_hedging_pnl(
            tf.constant(paths, tf.float32),
            tf.constant([self.params['K']]*n_paths, tf.float32),
            strategy
        ).numpy()

        # Analyse
        results = {
            'Black-Scholes': {
                'Moyenne': bs_pnl.mean(),
                'Écart-type': bs_pnl.std(),
                'CVaR 95%': self._calculate_cvar(bs_pnl, 0.95)
            },
            'LSTM': {
                'Moyenne': lstm_pnl.mean(),
                'Écart-type': lstm_pnl.std(),
                'CVaR 95%': self._calculate_cvar(lstm_pnl, 0.95)
            }
        }

        # Visualisation
        self._plot_results(bs_pnl, lstm_pnl)
        return results

    def _calculate_cvar(self, pnl, alpha):
        """Calcul du CVaR"""
        sorted_pnl = np.sort(pnl)
        n_worst = int((1 - alpha) * len(pnl))
        return -sorted_pnl[:n_worst].mean()

    def _plot_results(self, bs_pnl, lstm_pnl):
        """Visualisation des résultats"""
        plt.figure(figsize=(15, 6))

        # Distribution des PnL
        plt.subplot(1, 2, 1)
        sns.kdeplot(bs_pnl, label='Black-Scholes', fill=True)
        sns.kdeplot(lstm_pnl, label='LSTM', fill=True)
        plt.title('Distribution des PnL')
        plt.xlabel('PnL')
        plt.legend()

        # Comparaison des CVaR
        plt.subplot(1, 2, 2)
        plt.bar(['Black-Scholes', 'LSTM'],
                [self._calculate_cvar(bs_pnl, 0.95),
                 self._calculate_cvar(lstm_pnl, 0.95)])
        plt.title('CVaR 95% Comparaison')
        plt.ylabel('CVaR')

        plt.tight_layout()
        plt.show()

# ==================== EXÉCUTION ====================
def run_training():
    # Configuration
    params = {
        'S_0': 100,
        'T': 1/12,
        'r': 0.05,
        'vol': 0.2,
        'timesteps': 15,
        'n_sims': 10000,
        'batch_size': 500,
        'epochs': 5000,
        'alpha': 0.95
    }

    # Génération des données
    print("Génération des paths...")
    paths = monte_carlo_paths(
        S_0=params['S_0'],
        time_to_expiry=params['T'],
        sigma=params['vol'],
        drift=params['r'],
        seed=42,
        n_sims=params['n_sims'],
        n_timesteps=params['timesteps']
    )

    # Initialisation modèle
    model = Agent(
        time_steps=paths.shape[0],
        batch_size=params['batch_size'],
        features=1,
        T=params['T'],
        r=params['r']
    )

    # Entraînement
    print("Début de l'entraînement...")
    model.training(
        paths,
        np.full(params['n_sims'], 100),  # K=100
        params['alpha'],
        params['epochs']
    )

    return model

# Exécution principale
if __name__ == "__main__":
    drive.mount('/content/drive')
    trained_model = run_training()
    tester = HedgingTest()
    results = tester.compare_strategies(trained_model)
    print(pd.DataFrame(results).T)

Mounted at /content/drive
Génération des paths...
Début de l'entraînement...


Entraînement:   0%|          | 1/10000 [00:09<26:25:22,  9.51s/it]

Epoch 0 | CVaR: 7.5484


Entraînement:   0%|          | 11/10000 [00:19<3:08:48,  1.13s/it]

Epoch 10 | CVaR: 7.0361


...

Entraînement:  83%|████████▎ | 8251/10000 [2:24:00<28:18,  1.03it/s]

Epoch 8250 | CVaR: 3.6321


Entraînement:  83%|████████▎ | 8261/10000 [2:24:11<28:32,  1.02it/s]

Epoch 8260 | CVaR: 3.6267


Entraînement:  83%|████████▎ | 8266/10000 [2:24:17<34:03,  1.18s/it]

In [ ]:
import tensorflow as tf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import os
from tqdm import tqdm
from google.colab import drive
from scipy.stats import norm

# ==================== GÉNÉRATION DE DONNÉES ====================
def monte_carlo_paths(S_0, time_to_expiry, sigma, drift, seed, n_sims, n_timesteps):
    """Génère des paths de prix en 3D (n_timesteps+1, n_sims, 1)"""
    if seed is not None:
        np.random.seed(seed)

    dt_val = time_to_expiry / n_timesteps
    paths = np.zeros((n_timesteps + 1, n_sims, 1))
    paths[0, :, 0] = S_0

    for t in range(1, n_timesteps + 1):
        rand = np.random.normal(0, 1, n_sims)
        paths[t, :, 0] = paths[t-1, :, 0] * np.exp(
            (drift - 0.5*sigma**2)*dt_val +
            sigma*np.sqrt(dt_val)*rand
        )

    return paths

# ==================== MODÈLE LSTM ====================
class Agent(tf.keras.Model):
    def __init__(self, time_steps, batch_size, features, T, r=0.05, nodes=[64, 48, 32, 1], name='model'):
        super().__init__(name=name)

        self.time_steps = time_steps
        self.batch_size = batch_size
        self.features = features
        self.T = T  # Maturité
        self.r = r   # Taux sans risque

        # Architecture LSTM
        self.lstm_layers = [
            tf.keras.layers.LSTM(units,
                               return_sequences=True,
                               activation='tanh',
                               kernel_initializer='glorot_uniform')
            for units in nodes
        ]

        self.optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001,
            clipvalue=1.0
        )

    def call(self, inputs):
        x = inputs
        for lstm in self.lstm_layers:
            x = lstm(x)
        return tf.squeeze(x, axis=-1)

    def calculate_hedging_pnl(self, S_t_input, K, strategy):
        """Calcule le PnL avec gestion du cash"""
        S = tf.squeeze(S_t_input, axis=-1)
        time_steps = tf.shape(S)[0]
        dt_val = self.T / tf.cast(time_steps - 1, tf.float32)

        # Calcul des différences de delta
        delta_diff = tf.concat([
            strategy[0:1, :],  # Position initiale
            strategy[1:, :] - strategy[:-1, :]
        ], axis=0)

        # Simulation du cash
        cash = -delta_diff[0] * S[0]
        for t in range(1, time_steps-1):
            cash = cash * tf.exp(self.r * dt_val) - delta_diff[t] * S[t]

        # Valeur finale
        cash_final = cash * tf.exp(self.r * dt_val)
        Pi_T = strategy[-1] * S[-1] + cash_final
        return Pi_T - tf.maximum(S[-1] - K, 0)

    def calculate_cvar(self, pnl, alpha):
        """Calcule le CVaR différenciable"""
        sorted_pnl = tf.sort(pnl)
        n = tf.cast(tf.shape(pnl)[0], tf.float32)
        var_index = tf.cast((1 - alpha) * n, tf.int32)
        return -tf.reduce_mean(sorted_pnl[:var_index])

    @tf.function
    def train_step(self, S_t_input, K, alpha):
        """Étape d'entraînement optimisée"""
        with tf.GradientTape() as tape:
            strategy = self(S_t_input[:-1])
            pnl = self.calculate_hedging_pnl(S_t_input, K, strategy)
            cvar = self.calculate_cvar(pnl, alpha)

        grads = tape.gradient(cvar, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return cvar, pnl, strategy

    def training(self, paths, strikes, riskaversion, epochs):
        """Processus d'entraînement complet"""
        sample_size = paths.shape[1]
        idx = np.arange(sample_size)

        for epoch in tqdm(range(epochs), desc='Entraînement'):
            np.random.shuffle(idx)
            pnls = []

            for i in range(0, sample_size, self.batch_size):
                batch_idx = idx[i:i+self.batch_size]
                batch = paths[:, batch_idx, :]

                _, pnl, _ = self.train_step(
                    tf.cast(batch, tf.float32),
                    tf.cast(strikes[batch_idx], tf.float32),
                    tf.constant(riskaversion, tf.float32)
                )
                pnls.append(pnl.numpy())

            if epoch % 10 == 0:
                current_cvar = np.mean(-np.sort(np.concatenate(pnls))[:int(0.05*sample_size)])
                print(f"Epoch {epoch} | CVaR: {current_cvar:.4f}")

        return self

# ==================== ÉVALUATION ====================
class HedgingTest:
    def __init__(self, S_0=100, K=100, r=0.05, T=1/12, vol=0.2, timesteps=15):
        self.params = {
            'S_0': S_0,
            'K': K,
            'r': r,
            'T': T,
            'vol': vol,
            'timesteps': timesteps
        }
        self.dt = T / timesteps

    def black_scholes_delta(self, S, t):
        """Calcul du delta Black-Scholes corrigé"""
        if t <= 1e-6:
            return np.where(S > self.params['K'], 1.0, 0.0)

        d1 = (np.log(S/self.params['K'])
              + (self.params['r'] + 0.5*self.params['vol']**2)*t
             ) / (self.params['vol']*np.sqrt(t))

        return norm.cdf(d1)

    def calculate_bs_pnl(self, paths):
        """Calcul précis du PnL BS avec cash"""
        n_sims = paths.shape[1]
        pnls = np.zeros(n_sims)

        for i in range(n_sims):
            path = paths[:, i, 0]
            cash = 0.0
            delta_prev = 0.0

            for t in range(len(path)-1):
                time_left = self.params['T'] - t*self.dt
                delta = self.black_scholes_delta(path[t], time_left)
                cash = cash * np.exp(self.params['r'] * self.dt) - (delta - delta_prev)*path[t]
                delta_prev = delta

            cash_final = cash * np.exp(self.params['r'] * self.dt)
            Pi_T = delta_prev * path[-1] + cash_final
            pnls[i] = Pi_T - max(path[-1] - self.params['K'], 0)

        return pnls

    def compare_strategies(self, model, n_paths=5000):
        """Comparaison des stratégies corrigée"""
        # Génération des paths
        path_params = {
            'S_0': self.params['S_0'],
            'time_to_expiry': self.params['T'],
            'sigma': self.params['vol'],
            'drift': self.params['r'],
            'seed': 42,
            'n_sims': n_paths,
            'n_timesteps': self.params['timesteps']
        }
        paths = monte_carlo_paths(**path_params)

        # Calcul BS
        bs_pnl = self.calculate_bs_pnl(paths)

        # Calcul LSTM
        strategy = model(paths[:-1])
        lstm_pnl = model.calculate_hedging_pnl(
            tf.constant(paths, tf.float32),
            tf.constant([self.params['K']]*n_paths, tf.float32),
            strategy
        ).numpy()

        # Analyse
        results = {
            'Black-Scholes': {
                'Moyenne': bs_pnl.mean(),
                'Écart-type': bs_pnl.std(),
                'CVaR 95%': self._calculate_cvar(bs_pnl, 0.95)
            },
            'LSTM': {
                'Moyenne': lstm_pnl.mean(),
                'Écart-type': lstm_pnl.std(),
                'CVaR 95%': self._calculate_cvar(lstm_pnl, 0.95)
            }
        }

        # Visualisation
        self._plot_results(bs_pnl, lstm_pnl)
        return results

    def _calculate_cvar(self, pnl, alpha):
        """Calcul du CVaR"""
        sorted_pnl = np.sort(pnl)
        n_worst = int((1 - alpha) * len(pnl))
        return -sorted_pnl[:n_worst].mean()

    def _plot_results(self, bs_pnl, lstm_pnl):
        """Visualisation des résultats avec boxplots"""
        plt.figure(figsize=(18, 6))

        # 1. Distribution des PnL
        plt.subplot(1, 3, 1)
        sns.kdeplot(bs_pnl, label='Black-Scholes', fill=True, alpha=0.5)
        sns.kdeplot(lstm_pnl, label='LSTM', fill=True, alpha=0.5)
        plt.title('Distribution des PnL')
        plt.xlabel('PnL')
        plt.legend()

        # 2. Boxplots comparatifs
        plt.subplot(1, 3, 2)
        plt.boxplot([bs_pnl, lstm_pnl],
                    labels=['Black-Scholes', 'LSTM'],
                    patch_artist=True,
                    boxprops=dict(facecolor='lightblue', color='darkblue'),
                    medianprops=dict(color='red'))
        plt.title('Distribution des PnL par stratégie')
        plt.ylabel('PnL')
        plt.grid(True, alpha=0.3)

        # 3. Comparaison des CVaR
        plt.subplot(1, 3, 3)
        cvar_values = [self._calculate_cvar(bs_pnl, 0.95),
                      self._calculate_cvar(lstm_pnl, 0.95)]
        plt.bar(['Black-Scholes', 'LSTM'], cvar_values,
                color=['darkblue', 'orange'],
                alpha=0.7)
        plt.title('CVaR 95% Comparaison')
        plt.ylabel('CVaR')
        plt.grid(True, axis='y', alpha=0.3)

        plt.tight_layout()
        plt.show()

# ==================== EXÉCUTION ====================
def run_training():
    # Configuration
    params = {
        'S_0': 100,
        'T': 1/12,
        'r': 0.05,
        'vol': 0.2,
        'timesteps': 15,
        'n_sims': 10000,
        'batch_size': 500,
        'epochs': 100,
        'alpha': 0.95
    }

    # Génération des données
    print("Génération des paths...")
    paths = monte_carlo_paths(
        S_0=params['S_0'],
        time_to_expiry=params['T'],
        sigma=params['vol'],
        drift=params['r'],
        seed=42,
        n_sims=params['n_sims'],
        n_timesteps=params['timesteps']
    )

    # Initialisation modèle
    model = Agent(
        time_steps=paths.shape[0],
        batch_size=params['batch_size'],
        features=1,
        T=params['T'],
        r=params['r']
    )

    # Entraînement
    print("Début de l'entraînement...")
    model.training(
        paths,
        np.full(params['n_sims'], 100),  # K=100
        params['alpha'],
        params['epochs']
    )

    return model

# Exécution principale
if __name__ == "__main__":
    drive.mount('/content/drive')
    trained_model = run_training()
    tester = HedgingTest()
    results = tester.compare_strategies(trained_model)
    print(pd.DataFrame(results).T)